In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import numpy as np
import torch
import random

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('../base_model/Meta-Llama-3-8B-hf')

In [ ]:
tarin_set_id = random.randint(0, 1023)
test_set_id = random.randint(0, 7)

In [ ]:
raw_datasets = load_dataset('../dataset/c4/', data_files={'train': f'en/c4-train.{tarin_set_id}-of-01024.json.gz', 'test': f'en/c4-validation.{test_set_id}-of-00008.json.gz'})

In [ ]:
def process(example):
    ids = tokenizer.encode(example['text'])
    out = {'ids': ids, 'len': len(ids)}
    return out

tokenized = raw_datasets.map(process, desc='tokenizing raw datasets', num_proc=64)

In [ ]:
np.sum(tokenized['test']['len'])

In [ ]:
datasets = dict()

for split, dset in tokenized.items():
    datasets[split] = []
    length = np.sum(dset['len'])
    datasets[split] = np.ndarray((length, ), np.uint32)
    idx = 0
    for row in dset:
        datasets[split][idx:idx + row['len']] = row['ids']
        idx += row['len']

In [ ]:
torch.save(datasets, 'datasets.pt')